In [43]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


from datasets import load_dataset

dataset = load_dataset("cifar10")

# Organizando os Dadoas

In [44]:
X_train_png  = dataset['train']['img']
y_train      = dataset['train']['label']

X_test_png  = dataset['test']['img']
y_test      = dataset['test']['label']



In [45]:
X_train = []
X_test  = []
transf = transforms.ToTensor()


for x in X_train_png:
    prov = transf(x)
    X_train.append(prov)
    
X_train = X_train[0]

    
for x in X_test_png:
    prov = transf(x)
    X_test.append(prov)

X_test = X_test[0]


In [46]:
X_train

tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6667, 0.6588, 0.6471],
         [0.7059, 0.7020, 0.7059,  ..., 0.6784, 0.6706, 0.6588],
         [0.6941, 0.6941, 0.6980,  ..., 0.6706, 0.6627, 0.6549],
         ...,
         [0.4392, 0.4431, 0.4471,  ..., 0.3922, 0.3843, 0.3961],
         [0.4392, 0.4392, 0.4431,  ..., 0.4000, 0.4000, 0.4000],
         [0.4039, 0.3922, 0.4039,  ..., 0.3608, 0.3647, 0.3569]],

        [[0.6902, 0.6902, 0.6902,  ..., 0.6588, 0.6510, 0.6392],
         [0.6980, 0.6941, 0.6980,  ..., 0.6706, 0.6627, 0.6510],
         [0.6863, 0.6863, 0.6902,  ..., 0.6627, 0.6549, 0.6471],
         ...,
         [0.4196, 0.4275, 0.4314,  ..., 0.3804, 0.3686, 0.3725],
         [0.4000, 0.4039, 0.4039,  ..., 0.3725, 0.3647, 0.3608],
         [0.3765, 0.3647, 0.3725,  ..., 0.3294, 0.3373, 0.3294]],

        [[0.7412, 0.7412, 0.7412,  ..., 0.7059, 0.6941, 0.6824],
         [0.7490, 0.7451, 0.7490,  ..., 0.7137, 0.7059, 0.6941],
         [0.7373, 0.7373, 0.7412,  ..., 0.7059, 0.6980, 0.

# Dataset, DataLoader

In [47]:

class CIFAR10(Dataset):
    
    def __init__(self,X,y):
        
        
        self.X = X
        self.y = y
          
    def __getitem__(self,idx):
        return self.X[idx],self.y[idx]
    
    def __len__(self):
        return len(self.y)
    
    
    
# 

In [48]:
train_dl = DataLoader(CIFAR10(X_train,y_train),  batch_size = len(X_train) )
#test_dl  = DataLoader(CIFAR10(X_test_T, y_test_T),  batch_size = len(X_test_T) )
                          

# Criando o Modelo

In [50]:
class MLP_Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.L1  = nn.Linear(32*32,64)
        self.L2  = nn.Linear(64, 32)
        self.L3  = nn.Linear(32,10)
        self.A   = nn.ReLU()
        
        
    def forward(self):
        
        x = self.L1(x) 
        x = self.A(x)
        x = self.L2(x) 
        x = self.A(x)
        x = self.L2(x)
        x = self.A(x)
        
        return x
        

# Training

In [ ]:

def training( N_Epochs, model, loss_fn, opt  ):
    
    loss_list = []
    
    for epoch in tqdm(range(N_Epochs+1)):
        for xb, yb in train_dl:
            
            
            
            y_pred = model(xb.float())
            loss   = loss_fn(y_pred, yb.long())
        
            
            opt.zero_grad()
            loss.backward()
            opt.step()
        
            loss_list.append(loss.item())
    
    L1_weights = MLP_Model().L1.weight.detach()
    L2_weights = MLP_Model().L2.weight.detach()
    L3_weights = MLP_Model().L3.weight.detach()
    L4_weights = MLP_Model().L4.weight.detach()
    
       
    
    plt.figure(figsize = (14,6))
    plt.title("cost decay")
    plt.plot(loss_list)
    plt.xlabel("epoch")
    plt.ylabel("cost")
    
    return L1_weights, L2_weights, L3_weights, L4_weights